In [ ]:
# STEP 3 — MERGE + SALIENCE
# Inputs: ip_posts.csv, ip_comments.csv
# Outputs: merged_posts_comments.csv, daily_metrics.csv, weekly_metrics.csv

# inspired by https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html

import pandas as pd

# Inputs from Steps 1–2 (CSV-only)
POSTS_CSV    = "ip_posts.csv"
COMMENTS_CSV = "ip_comments.csv"

# Outputs (CSV-only)
POSTS_CLEAN_CSV    = "posts_clean.csv"
COMMENTS_CLEAN_CSV = "comments_clean.csv"
MERGED_CSV         = "merged_posts_comments.csv"
DAILY_METRICS_CSV  = "daily_metrics.csv"
WEEKLY_METRICS_CSV = "weekly_metrics.csv"

# Date window
# Keep 2020-01-01 .. 2025-09-01 inclusive
START_DT_UTC = pd.Timestamp("2020-01-01", tz="UTC")
END_DT_UTC   = pd.Timestamp("2025-09-02", tz="UTC")  # exclusive upper bound

# Naive versions for date-based indices
START_NAIVE = pd.Timestamp("2020-01-01")
END_NAIVE   = pd.Timestamp("2025-09-02")  # exclusive

# Load
posts = pd.read_csv(POSTS_CSV, dtype={"id":"string"}, low_memory=False)
comments = pd.read_csv(COMMENTS_CSV, dtype={"id":"string","submission_id":"string"}, low_memory=False)

def to_dt_utc_from_epoch(series):
    return pd.to_datetime(series, unit="s", errors="coerce", utc=True)

def clip_window_utc(df, dtcol="dt"):
    """Row-level clip with tz-aware UTC bounds."""
    return df[(df[dtcol] >= START_DT_UTC) & (df[dtcol] < END_DT_UTC)].copy()

# Clean POSTS
p = posts.copy()
p["kind"] = "post"
p["dt"] = pd.to_datetime(p["dt"], errors="coerce", utc=True) if "dt" in p.columns else to_dt_utc_from_epoch(p["created_utc"])
p = p[p["dt"].notna()].copy()
p = clip_window_utc(p, "dt")
p["date"] = p["dt"].dt.date
p.rename(columns={"link_flair_text": "flair"}, inplace=True)
p["title"] = p.get("title", "").fillna("")
p["selftext"] = p.get("selftext", "").fillna("")
p["text"] = (p["title"].astype(str).str.strip() + " " + p["selftext"].astype(str).str.strip()).str.strip()

post_keep = [
    "id","kind","created_utc","dt","date","subreddit","author","title","selftext",
    "score","num_comments","flair","url","permalink","text"
]
p = p[[c for c in post_keep if c in p.columns]].copy()
p.to_csv(POSTS_CLEAN_CSV, index=False)

# Clean COMMENTS
c = comments.copy()
c["kind"] = "comment"
c["dt"] = pd.to_datetime(c["dt"], errors="coerce", utc=True) if "dt" in c.columns else to_dt_utc_from_epoch(c["created_utc"])
c = c[c["dt"].notna()].copy()
c = clip_window_utc(c, "dt")
c["date"] = c["dt"].dt.date
if "submission_flair" in c.columns and "flair" not in c.columns:
    c.rename(columns={"submission_flair":"flair"}, inplace=True)
c["body"] = c.get("body","").fillna("")
c["text"] = c["body"].astype(str)

com_keep = [
    "id","kind","created_utc","dt","date","subreddit","author","body",
    "score","link_id","parent_id","submission_id","flair","text","permalink"
]
c = c[[col for col in com_keep if col in c.columns]].copy()
c.to_csv(COMMENTS_CLEAN_CSV, index=False)

# Merge (long format)
merged = pd.concat([p, c], ignore_index=True, sort=False)
common_cols = ["id","kind","created_utc","dt","date","subreddit","author","score","flair","text"]
extra_cols  = [col for col in merged.columns if col not in common_cols]
merged = merged[common_cols + extra_cols]
merged.to_csv(MERGED_CSV, index=False)

# Daily & weekly volumes
daily = (merged.groupby(["date","kind"])
                .size()
                .unstack("kind")
                .fillna(0)
                .rename(columns={"post":"posts","comment":"comments"}))

# Clip with tz-naive bounds (daily index is naive)
daily_index_dt = pd.to_datetime(daily.index)
mask = (daily_index_dt >= START_NAIVE) & (daily_index_dt < END_NAIVE)
daily = daily.loc[mask]
daily["total"] = daily.sum(axis=1)
daily = daily.sort_index()
daily.to_csv(DAILY_METRICS_CSV)

weekly = (daily.set_index(pd.to_datetime(daily.index))
               .resample("W-MON").sum()
               .rename_axis("week_start"))

# Clip weekly (naive index)
weekly = weekly[(weekly.index >= START_NAIVE) & (weekly.index < END_NAIVE)]
weekly.to_csv(WEEKLY_METRICS_CSV)

print(f"[ok] posts_clean → {POSTS_CLEAN_CSV} ({len(p):,} rows)")
print(f"[ok] comments_clean → {COMMENTS_CLEAN_CSV} ({len(c):,} rows)")
print(f"[ok] merged → {MERGED_CSV} ({len(merged):,} rows)")
print(f"[ok] metrics → {DAILY_METRICS_CSV}, {WEEKLY_METRICS_CSV}")

try:
    display(daily.head(3)); display(daily.tail(3))
    display(weekly.head(3)); display(weekly.tail(3))
except Exception:
    pass




[ok] posts_clean → posts_clean.csv (2,176 rows)
[ok] comments_clean → comments_clean.csv (483,216 rows)
[ok] merged → merged_posts_comments.csv (485,392 rows)
[ok] metrics → daily_metrics.csv, weekly_metrics.csv


kind,comments,posts,total
date,,,
2020-05-07,27.0,1.0,28.0
2020-05-08,4.0,0.0,4.0
2020-05-14,2.0,0.0,2.0


kind,comments,posts,total
date,,,
2025-08-30,2377.0,38.0,2415.0
2025-08-31,2226.0,27.0,2253.0
2025-09-01,2563.0,33.0,2596.0


kind,comments,posts,total
week_start,,,
2020-05-11,31.0,1.0,32.0
2020-05-18,2.0,0.0,2.0
2020-05-25,181.0,1.0,182.0


kind,comments,posts,total
week_start,,,
2025-08-18,15896.0,128.0,16024.0
2025-08-25,20229.0,171.0,20400.0
2025-09-01,18609.0,229.0,18838.0
